<a href="https://colab.research.google.com/github/prarthanasigedar/Semantic-Segmentation-for-brain-tumors-U-Net/blob/master/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
pip install nilearn

     |████████████████████████████████| 2.5MB 28kB/s 


In [ ]:
#import tensorflow as tf
#from tensorflow import keras
import matplotlib.pyplot as plt
import glob
import nibabel as nib
from nilearn import plotting
from keras.layers import Dense,Conv2D,Activation,Input,Flatten,Dropout,merge,Reshape,MaxPooling2D,UpSampling2D,Conv2DTranspose,Add,AveragePooling2D,Concatenate,SpatialDropout2D
from keras.layers.normalization import BatchNormalization 
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.initializers import glorot_uniform
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras.utils import *
from keras import regularizers
from keras import backend as K
import numpy as np
from keras.models import load_model
import pickle

Using TensorFlow backend.


In [ ]:
pip install SimpleITK

     |████████████████████████████████| 42.5MB 72kB/s 


In [ ]:
import SimpleITK as sitk

In [ ]:
flair_path = glob.glob('/content/drive/My Drive/HGG/flair/*')
seg_path = glob.glob('/content/drive/My Drive/HGG/seg/*')
t1_path = glob.glob('/content/drive/My Drive/HGG/t1/*')
t1ce_path = glob.glob('/content/drive/My Drive/HGG/t1ce/*')
t2_path   = glob.glob('/content/drive/My Drive/HGG/t2/*')

In seg file
--------------
Label 1: necrotic and non-enhancing tumor ; 
Label 2: edema ; 
Label 4: enhancing tumor; 
Label 0: background ; 

MRI
-------
whole/complete tumor: 1 2 4 ;
core: 1 4 ;
enhance: 4 ;

In [ ]:
print(len(flair_path))

259


1)Appending all the alternate slices of each modality in X_flair, X_t1, X_t1ce ,X_t2 and X_seg.



2) Converting them into array and then normalization 


In [ ]:
X_flair=[]
for i in range(len(flair_path)):
    a = sitk.ReadImage(flair_path[i])
    a = sitk.GetArrayFromImage(a)
    for j in range(25):
        X_flair.append((a[50+2*j,:,:]))
print(np.shape(X_flair))

(6475, 240, 240)


In [ ]:
X_flair = np.asarray(X_flair).astype(np.float32)
print(X_flair.shape)

(6475, 240, 240)


In [ ]:
mean = X_flair.mean()
std = X_flair.std()
X_flair = (X_flair - mean)/(std + 1e-8)
np.min(X_flair), np.max(X_flair)

(-0.46841258, 16.260946)

In [ ]:
X_t1=[]
for i in range(len(t1_path)):
    a = sitk.ReadImage(t1_path[i])
    a = sitk.GetArrayFromImage(a)
    for j in range(25):
        X_t1.append((a[50+2*j,:,:]))
print(np.shape(X_t1))

(6475, 240, 240)


In [ ]:
X_t1 = np.asarray(X_t1).astype(np.float32)
print(X_t1.shape)

(6475, 240, 240)


In [ ]:
mean = X_t1.mean()
std = X_t1.std()
X_t1 = (X_t1 - mean)/(std + 1e-8)
np.min(X_t1), np.max(X_t1)

(-0.41608155, 24.9923)

In [ ]:
X_t1ce=[]
for i in range(len(t1ce_path)):
    a = sitk.ReadImage(t1ce_path[i])
    a = sitk.GetArrayFromImage(a)
    for j in range(25):
        X_t1ce.append((a[50+2*j,:,:]))
print(np.shape(X_t1ce))

(6475, 240, 240)


In [ ]:
X_t1ce = np.asarray(X_t1ce).astype(np.float32)
print(X_t1ce.shape)

(6475, 240, 240)


In [ ]:
mean = X_t1ce.mean()
std = X_t1ce.std()
X_t1ce = (X_t1ce - mean)/(std + 1e-8)
np.min(X_t1ce), np.max(X_t1ce)

(-0.4047665, 31.285044)

In [ ]:
X_t2=[]
for i in range(len(t2_path)):
    a = sitk.ReadImage(t2_path[i])
    a = sitk.GetArrayFromImage(a)
    for j in range(25):
        X_t2.append((a[50+2*j,:,:]))
print(np.shape(X_t2))

(6475, 240, 240)


In [ ]:
X_t2 = np.asarray(X_t2).astype(np.float32)
print(X_t2.shape)

(6475, 240, 240)


In [ ]:
mean = X_t2.mean()
std = X_t2.std()
X_t2 = (X_t2 - mean)/(std + 1e-8)
np.min(X_t2), np.max(X_t2)

(-0.42582026, 35.94511)

In [ ]:
X_seg=[]
for i in range(len(seg_path)):
    a = sitk.ReadImage(seg_path[i])
    a = sitk.GetArrayFromImage(a)
    for j in range(25):
        X_seg.append((a[50+2*j,:,:]))
print(np.shape(X_seg))

(6475, 240, 240)


In [ ]:
X_seg = np.asarray(X_seg).astype(np.float32)

In [ ]:
data = np.zeros((X_flair.shape[0],240,240,4))
for i in range(data.shape[0]):
    data[i,:,:,0] = X_flair[i,:,:]
    data[i,:,:,1] = X_t1[i,:,:]
    data[i,:,:,2] = X_t1ce[i,:,:]
    data[i,:,:,3] = X_t2[i,:,:]

In [ ]:
print(np.shape(data))

(6475, 240, 240, 4)


In [ ]:
# to save the memory delete the variables later
del X_flair
del X_t1
del X_t2
del X_t1ce

In [ ]:
print(np.unique(X_seg))

[0. 1. 2. 4.]


In [ ]:
X_seg[X_seg == 4] = 3

In [ ]:
print(np.unique(X_seg))

[0. 1. 2. 3.]


In [ ]:
print(X_seg.shape)

(6475, 240, 240)


In [ ]:
gt = to_categorical(X_seg, num_classes=None)
print(gt.shape)

(6475, 240, 240, 4)


In [ ]:
print(data.shape)
print(gt.shape)

(6475, 240, 240, 4)
(6475, 240, 240, 4)


In [ ]:
del X_seg

In [ ]:
#splitting the dataset into training, testing and validation data
X_train = data[:4700]
X_valid = data[4700:5475]
X_test  = data[5475:]

In [ ]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(4700, 240, 240, 4)
(775, 240, 240, 4)
(1000, 240, 240, 4)


In [ ]:
Y_train = gt[:4700]
Y_valid = gt[4700:5475]
Y_test  = gt[5475:]

In [ ]:
print(Y_train.shape)
print(Y_valid.shape)
print(Y_test.shape)

(4700, 240, 240, 4)
(775, 240, 240, 4)
(1000, 240, 240, 4)


PREPROCESSING OF DATA

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_datagen = ImageDataGenerator(rotation_range=10,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    shear_range=0.1,
                                    zoom_range=0.1)
mask_datagen = ImageDataGenerator(rotation_range=10,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    shear_range=0.1,
                                    zoom_range=0.1)

In [ ]:
seed = 1

In [ ]:
image_generator = image_datagen.flow(X_train, batch_size=12, seed=seed)

In [ ]:
mask_generator = mask_datagen.flow(Y_train, batch_size=12, seed=seed)

In [ ]:
train_generator = zip(image_generator, mask_generator)

In [ ]:
batch_size = 12
image_size = 240
epochs = 100
inChannel = 4
x, y = 240, 240
input_img = Input(shape = (x, y, inChannel))